In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
from misic.misic import *
from misic.extras import *
from skimage.io import imsave,imread
from skimage.transform import resize,rescale

import time, os, sys
import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import edt

In [ ]:
def diameters(masks,dist_threshold=1):
    dt = edt.edt(np.int32(masks))
    dt_pos = np.abs(dt[dt>=dist_threshold])
    return dist_to_diam(np.abs(dt_pos))
def dist_to_diam(dt_pos):
    return 6*np.mean(dt_pos)

In [ ]:
# tesr dataset
from pathlib import Path
from cellpose_omni import io

basedir = '/home/kcutler/DataDrive/ecVSsp/'
phase = Path(basedir).rglob('*c1.tif')
img_names = [str(p) for p in phase]
imgs = [skimage.io.imread(f) for f in img_names]

In [ ]:
def getname(path,suffix=''):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

In [ ]:
from skimage.transform import rescale,resize
from skimage.util import random_noise
from skimage.filters import gaussian
from cellpose_omni import transforms

import cv2
Y = []
# Initialize MiSiC
do_size = 0
invert = 0
mseg = MiSiC(model_dir='/home/kcutler/DataDrive/misic_etc/new_misic_model_kevin_ep100_bs8.h5')
# mseg = MiSiC(model_dir='/home/kcutler/DataDrive/misic_etc/new_misic_model_kevin.h5')
# mseg = MiSiC()
for idx in range(len(imgs)):

# for idx in [1,77,100,101]:
    
    im = imgs[idx].copy()
    sr,sc = im.shape
    
    # added per Swapnesh's recommendations, helps a lot, holy cow! 
    # blur not so much, bit a tiny bit of noise is good
    if 1:
        # im = gaussian(im,1.5)
        im = transforms.normalize99(im,skel=True)
        im = random_noise(im,mode ='gaussian',var = 0.000001)
        # im = add_noise(im, sensitivity = .1, invert = invert)

    if do_size: # my custom model not trained with resizing in mind, variance in width is small 
        #input the approximate mean width of microbe under consideration
        mean_width = diams[idx]/2
        # compute scaling factor
        scale = (10/mean_width)
        # preprocess using inbuit function or if you are feeling lucky use your own preprocessing
        im = rescale(im,scale,preserve_range = True)
    
    
    # Parameters that need to be changed
    ## Ideally, use a single image to fine tune two parameters : mean_width and noise_variance (optional)
    # add local noise
    # img = add_noise(im, sensitivity = .5, invert = True)
    img = im.copy()

    # segment
    yp = mseg.segment(img,invert = invert)
    # yp = mseg.segment_auto(im) broken
    if do_size:
        yp = resize(yp,(sr,sc))
        img = resize(img,(sr,sc))
    masks  = postprocess_ws(cv2.resize(img,[yp.shape[1],yp.shape[0]]),yp)
    Y.append(masks)
    plt.imshow(np.hstack((img,yp[:,:,0],yp[:,:,1])))
    plt.axis('off')
    plt.show()
    plt.imshow(masks)
    plt.axis('off')
    plt.show()
    savedir = '/home/kcutler/DataDrive/omnipose_paper/Figure 6/MiSiC/'
    io.check_dir(savedir)
    io.imsave(os.path.join(savedir,getname(img_names[idx])+'_masks.tif'),masks)

In [ ]:
# plt.imshow(masks)
# yp.shape,im.shape,img.shape
# m_orig = masks.copy()
np.any(masks-m_orig)
# %reset_selective -f mseg
# mseg

In [ ]:
im = np.array(imgs[0].copy())
print(im,im.shape)
# im = im.resize()
# im.shape
im = im.resize((int(im.shape[0]*scale),int(im.shape[1]*scale)))
print(im, im.shape)

In [ ]:
from cellpose_omni import utils
utils.diameters(masks,skel=1)

In [ ]:
# body = resize(yp[:,:,0],[sr,sc])
# contours = resize(yp[:,:,1],[sr,sc])

# watershed based post processing (optional)


In [ ]:
plt.imshow(yp[:,:,0])

In [ ]:
# save 8-bit segmented image and use it as you like
imsave('segmented.tif', yp.astype(np.uint8))
''''

### In case of gpu error, one might need to disabple gpu before importing MiSiC [ os.environ["CUDA_VISIBLE_DEVICES"]="-1" ]

About

segmentation of rod-shaped bacteria
Resources
Readme
License
GPL-3.0 License
Releases 2
MiSiC-v1 Latest
on Apr 4
+ 1 release
Packages
No packages published
Languages

Python 100.0%

In [ ]:
mseg = MiSiC(model_dir='/home/kcutler/DataDrive/misic_etc/new_misic_model_kevin_hist.h5')
mseg_orig = MiSiC()
mseg.model.trainable_variables[0][0][0]==mseg_orig.model.trainable_variables[0][0][0]

In [ ]:
mseg,mseg_orig

In [ ]:
mseg,mseg_orig